In [ ]:
import tensorflow as tf
import numpy as np
import cv2
import os
import shutil

from tensorflow import keras
from matplotlib import pyplot as plt

# Chargement du modèle

On commence par charger notre modèle en mémoire :

In [ ]:
import classification_models
from classification_models.tfkeras import Classifiers

# Chargement du modèle ResNEt18
ResNet18, preprocess_input = Classifiers.get('resnet18')

# Instanciation du modèle pré-entrainé ResNet18
base_model = ResNet18(input_shape=(224,224,3), weights='imagenet', include_top=False,pooling=False)

# Désactivation des couches pour l'entrainement
for layer in base_model.layers:
    layer.trainable = False
    
# Ajout de l'applatissement des sorties et de la couche dense avec 2 neurones"
x = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)
output = tf.keras.layers.Dense(units=2, activation='softmax')(x)

model = tf.keras.Model(inputs=[base_model.input], outputs=[output])

In [ ]:
model.load_weights("meilleur_modele_colab.hdf5")

# Exportation du modèle au format Tensorflow

In [ ]:
tf.saved_model.save(model,"meilleur_model_colab")

Pour obtenir des informations sur le modèlé on peut utiliser la commande **saved_model_cli** :

In [ ]:
!saved_model_cli show --all --dir meilleur_model_colab

# Conversion du modèle sauvegardé au format ONNX

Nous aurons besoin par la suite du modèle au format ONNX pour réaliser les optimisations.

**Remarque importante** : Si vous voyez le message ***IMPORTANT*** Installed protbuf is not cpp accelerated... Il faut mettre à jour votre version en utilisant le script disponible ici : https://github.com/AlexandreBourrieau/JetsonNano/blob/main/install_protobuf-3.19.4.sh

In [ ]:
!python3 -m tf2onnx.convert --saved-model "meilleur_model_colab" --output "meilleur_model_colab/meilleur_model_colab.onnx"

On modifie maintenant le batch_size :

In [ ]:
import onnx

batch_size = 1
onnx_model = onnx.load("meilleur_model_colab/meilleur_model_colab.onnx")

Regardons à quoi ressemble l'entrée du modèle :

In [ ]:
inputs = onnx_model.graph.input

In [ ]:
inputs

In [ ]:
batch_size = 1

inputs = onnx_model.graph.input
for input in inputs:
    dim1 = input.type.tensor_type.shape.dim[0]
    dim1.dim_value = batch_size

Puis on sauvegarde les modifications :

In [ ]:
onnx.save_model(onnx_model,"meilleur_model_colab/meilleur_model_colab.onnx")